# Рекомендательные системы

<b><i>Описание задачи</i></b>

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать <b>recall@k</b> и <b>precision@k</b>.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

<b><i>Входные данные</i></b>

Вам дается две выборки с пользовательскими сессиями - <b>id</b>-шниками просмотренных и <b>id</b>-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, <b>1,2,3,4;</b> или <b>1,2,3,4;5,6.</b>

Гарантируется, что среди <b>id</b> купленных товаров все различные.

<b><i>Важно:</b></i>

1. Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
2. Если товар не встречался в обучающей выборке, его популярность равна 0.
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и <b>k</b> в <b>recall@k</b> / <b>precision@k</b>.

<b><i>Задание</b></i>

1. На обучении постройте частоты появления <b>id</b> в просмотренных и в купленных (<b>id</b> может несколько раз появляться в просмотренных, все появления надо учитывать)

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data_file = open('coursera_sessions_train.txt', 'r')

In [3]:
goods_dict = {}
for session_str in train_data_file:
    view_goods_id_in_session_str, buy_goods_id_in_session_str = str.split(session_str[:-1], ';')
    view_goods_id_in_session_list = str.split(view_goods_id_in_session_str, ',')
    for view_good_id in view_goods_id_in_session_list:
        if len(view_good_id) > 0:
            view_good_id = np.int32(view_good_id)
            if view_good_id not in goods_dict:
                goods_dict.update({view_good_id: [0, 0]})
            goods_dict[view_good_id][0] += 1
        
    buy_goods_id_in_session_list = str.split(buy_goods_id_in_session_str, ',')
    for buy_good_id in buy_goods_id_in_session_list:
        if len(buy_good_id) > 0:
            buy_good_id = np.int32(buy_good_id)
            if buy_good_id not in goods_dict:
                goods_dict.update({buy_good_id: [0, 0]})
            goods_dict[buy_good_id][1] += 1

In [4]:
goods_pop_table = {'id': np.array(goods_dict.keys()),
                   'view_num': np.array(goods_dict.values())[:, 0],
                   'buy_num': np.array(goods_dict.values())[:, 1]}
goods_pop_df = pd.DataFrame(goods_pop_table)
goods_pop_df = goods_pop_df[['id', 'view_num', 'buy_num']]

In [5]:
goods_pop_df.head(10)

,id,view_num,buy_num
0,0,6,0
1,1,6,0
2,2,9,0
3,3,7,0
4,4,11,0
5,5,4,1
6,6,283,2
7,7,312,2
8,8,225,3
9,9,7,0


2. Реализуйте два алгоритма рекомендаций:
    * сортировка просмотренных <b>id</b> по популярности (частота появления в просмотренных),
    * сортировка просмотренных <b>id</b> по покупаемости (частота появления в покупках).

In [6]:
def to_do_recommend_list(view_goods_id_list, goods_dict, max_recommend_num, recommend_algorithm='view'):
#     unique_view_goods_id_in_session_list = list(set(view_goods_id_list))
    unique_view_goods_id_in_session_list = []
    for val in view_goods_id_list:
        if val not in unique_view_goods_id_in_session_list:
            unique_view_goods_id_in_session_list.append(val)
    view_goods_data = []
    for i, view_good_id in enumerate(unique_view_goods_id_in_session_list):
        if view_good_id in goods_dict:
            view_buy_num = goods_dict[view_good_id]
            view_goods_data.append([view_good_id, view_buy_num[0], view_buy_num[1], i])
        else:
            view_goods_data.append([view_good_id, 0, 0, i])
    if recommend_algorithm == 'view':
        sorted_view_goods_data = sorted(view_goods_data, key = lambda x: (x[1], -x[3]))
    else:
        sorted_view_goods_data = sorted(view_goods_data, key = lambda x: (x[2], -x[3]))
    if len(sorted_view_goods_data) <= max_recommend_num:
        recommended_goods = np.array(sorted_view_goods_data)[:, 0]
        A = np.array(sorted_view_goods_data)[:]
    else:
        recommended_goods = np.array(sorted_view_goods_data)[-max_recommend_num:, 0]
        A = np.array(sorted_view_goods_data)[-max_recommend_num:]
    return recommended_goods

In [7]:
def to_calc_metrics_with_view_recommend(data_file, goods_dict, k, recommend_algorithm='view'):
    precisions = []
    recalls = []
    for session_str in data_file:
        session_str = session_str[:-1]
        view_goods_id_in_session_str, buy_goods_id_in_session_str = str.split(session_str, ';')
        if buy_goods_id_in_session_str is not '':
            view_goods_id_in_session_list = np.int32(np.array(str.split(view_goods_id_in_session_str, ',')))
            buy_goods_id_in_session_list = np.int32(np.array(str.split(buy_goods_id_in_session_str, ',')))
            recommended_goods = to_do_recommend_list(view_goods_id_in_session_list, goods_dict, k, recommend_algorithm=recommend_algorithm)
            bought_from_recommended = 0
            for bought_goods in buy_goods_id_in_session_list:
                if bought_goods in recommended_goods:
                    bought_from_recommended += 1
#             precision = bought_from_recommended / len(recommended_goods)
            precision = bought_from_recommended / float(k)
            precisions.append(precision)
            recall = bought_from_recommended / float(len(buy_goods_id_in_session_list))
            recalls.append(recall)
    average_precision = np.array(precisions).mean()
    average_recall = np.array(recalls).mean()
    return round(average_recall, 2), round(average_precision, 2)

3. Для данных алгоритмов выпишите через пробел <b>AverageRecall@1</b>, <b>AveragePrecision@1</b>, <b>AverageRecall@5</b>, <b>AveragePrecision@5</b> на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

In [8]:
train_data_file = open('coursera_sessions_train.txt', 'r')
to_calc_metrics_with_view_recommend(train_data_file, goods_dict, 1, recommend_algorithm='view')

(0.44, 0.51)

In [9]:
train_data_file = open('coursera_sessions_train.txt', 'r')
to_calc_metrics_with_view_recommend(train_data_file, goods_dict, 5, recommend_algorithm='view')

(0.82, 0.21)

In [10]:
test_data_file = open('coursera_sessions_test.txt', 'r')
to_calc_metrics_with_view_recommend(test_data_file, goods_dict, 1, recommend_algorithm='view')

(0.42, 0.48)

In [11]:
test_data_file = open('coursera_sessions_test.txt', 'r')
to_calc_metrics_with_view_recommend(test_data_file, goods_dict, 5, recommend_algorithm='view')

(0.8, 0.2)

In [12]:
train_data_file = open('coursera_sessions_train.txt', 'r')
to_calc_metrics_with_view_recommend(train_data_file, goods_dict, 1, recommend_algorithm='buy')

(0.69, 0.8)

In [13]:
train_data_file = open('coursera_sessions_train.txt', 'r')
to_calc_metrics_with_view_recommend(train_data_file, goods_dict, 5, recommend_algorithm='buy')

(0.93, 0.25)

In [14]:
test_data_file = open('coursera_sessions_test.txt', 'r')
to_calc_metrics_with_view_recommend(test_data_file, goods_dict, 1, recommend_algorithm='buy')

(0.46, 0.53)

In [15]:
test_data_file = open('coursera_sessions_test.txt', 'r')
to_calc_metrics_with_view_recommend(test_data_file, goods_dict, 5, recommend_algorithm='buy')

(0.82, 0.21)

In [16]:
view_goods_data = [[1, 3, 10], [2, 2, 5], [3, 7, 5], [4, 3, 5], [5, 6, 10], [6, 2, 4], [7, 6, 11], [8, 2, 8], [9, 7, 10], [10, 5, 1]]
A = sorted(view_goods_data, key = lambda x: (x[1], -x[2]))
A

[[8, 2, 8],
 [2, 2, 5],
 [6, 2, 4],
 [1, 3, 10],
 [4, 3, 5],
 [10, 5, 1],
 [7, 6, 11],
 [5, 6, 10],
 [9, 7, 10],
 [3, 7, 5]]

In [17]:
np.array(A)[:, 0]

array([ 8,  2,  6,  1,  4, 10,  7,  5,  9,  3])

In [18]:
set([1, 1, 2, 5, 3, 2, 10, 1, 5])

{1, 2, 3, 5, 10}

In [19]:
A = [1, 1, 2, 5, 3, 2, 10, 1, 5, -3, 7]
B = []
for a in A:
    if a not in B:
        B.append(a)

In [20]:
B

[1, 2, 5, 3, 10, -3, 7]

<b><i>Дополнительные вопросы</b></i>

1. Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
2. Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.